In [1]:
import re
import numpy as np
SPAM = 1
NOT_SPAM = 0

class Filter():
    def __init__(self):
        self.pA = 0
        self.pnotA = 0
        self.dict_spam = {}
        self.dict_no_spam = {}
        self.SPAM = 1
        self.NOT_SPAM = 0
        self.word_spam = 0
        self.word_no_spam = 0
        self.mail_spam = 0
        self.no_spam = 0
        self.spam = 0
        
        
    def train(self, data):
        for  mail, label in data:
            self.calculate_word_frequencies(mail, label)
            if label == SPAM:
                self.mail_spam += 1
        total_mail = len(data)
        self.pA = self.mail_spam / total_mail
        self.pnotA = 1 - self.pA
        
    def calculate_word_frequencies(self, body, lab):
        for i in re.findall(r"\w+",body):
            
            if lab == SPAM:
                self.dict_spam[i.lower()] = self.dict_spam.get(i.lower(),0) + 1
                self.word_spam+=1
            else:
                self.dict_no_spam[i.lower()] = self.dict_no_spam.get(i.lower(),0) + 1
                self.word_no_spam+=1

    def calculate_P_Bi_A(self, word, label):
#         vocabulaty_size = len(set(self.dict_spam) | set(self.dict_no_spam)) #сглаживание, 
        if label == SPAM:
            return (self.dict_spam.get(word,0) + 1)/self.word_spam
#             return (self.dict_spam.get(word,0) + 1)/self.word_spam+1*vocabulaty_size
        else:
            return (self.dict_no_spam.get(word,0) + 1)/self.word_no_spam
#             return (self.dict_no_spam.get(word,0) + 1)/self.word_no_spam+1*vocabulaty_size
    
    def calculate_P_B_A(self, text, label):
        result = 1
        for i in re.findall(r"\w+",text):
            result += np.log(self.calculate_P_Bi_A(i.lower(), label))
        return result

    
    def classify(self, email):
        self.spam = np.log(self.pA)+self.calculate_P_B_A(email, SPAM)
        self.no_spam = np.log(self.pnotA)+self.calculate_P_B_A(email, NOT_SPAM)
        if self.spam > self.no_spam:
            return SPAM
        else:
            return NOT_SPAM

In [2]:
import pandas as pd
df = pd.read_csv("spam_or_not_spam.csv")

In [3]:
df.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [4]:
df = df.dropna()
df['label'] = df['label'].apply(lambda x: SPAM if x==1 else 0)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, df['label'], test_size=0.30, random_state=42)

In [5]:
X_train.label.value_counts(normalize=True)

0    0.838018
1    0.161982
Name: label, dtype: float64

In [6]:
X_test.label.value_counts(normalize=True)

0    0.823333
1    0.176667
Name: label, dtype: float64

In [7]:
X_test.shape

(900, 2)

In [8]:
data_list = X_train.values.tolist()

In [9]:
data = X_test.values.tolist()

In [10]:
test=Filter()

In [11]:
test.train(data_list)

In [12]:
%%time
X_train['prediction'] = X_train.apply(lambda row: test.classify(row['email']), axis=1)
X_test['prediction'] = X_test.apply(lambda row: test.classify(row['email']), axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Wall time: 15.9 s


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
X_test.head()

,email,label,prediction
1376,on wednesday NUMBER august NUMBER NUMBER NUMBE...,0,0
932,on NUMBER aug NUMBER at NUMBER NUMBER rohit kh...,0,0
144,url URL date NUMBER NUMBER NUMBERtNUMBER NUMBE...,0,0
1752,on NUMBER september NUMBER tim peters said why...,0,0
51,begin pgp signed message justin maccarthy wro...,0,0


In [14]:
from sklearn.metrics import f1_score
score = f1_score(X_test['label'], X_test['prediction'])
print('f1_score of model prediction:', score)

f1_score of model prediction: 0.9483282674772036


In [15]:
from sklearn.metrics import confusion_matrix
score = confusion_matrix(X_test['label'], X_test['prediction'])
print(score)

[[727  14]
 [  3 156]]
